In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_mypy

In [ ]:
import nbimporter

# Vorbereitung
Alle Notebooks wurden unter Python mit der Version `3.10.1` erstellt. Im Folgenden können die Version und die Pakete überprüft und ggf. installiert werden.


In [ ]:
from platform import python_version
print("Required Python version:", "3.10.1")
print("Your Python version:\t", python_version())

# Setup commands
# conda create -n chess python=3.10.1 -c conda-forge
# conda activate chess
# pip install -r src/requirements.txt
# conda install jupyter -c conda-forge
# jupyter notebook

# Chess-AI

Dieses Notebook führt alle einzelnen Dateien und Klassen zusammen und implementiert den Spielablauf einer Partie.
Folgende Dateien werden in dieser Reihenfolge importiert:
- `AI-Base-Class.ipynb`: Die abstrakte Basisklasse für alle AI-Versionen, welche das Eröffnungs- und Endspiel implementiert.
- `Exercise01AI`: Die erste Version der AI, welche im Mittelspiel zufällige Züge auswählt.

Anschließend wird das Spiel durchgeführt, mit der Möglichkeit die Partie danach auf der Festplatte zu speichern.

In [ ]:
from AIBaseClass import ChessAI, State
from Exercise01AI import Exercise01AI
from HumanPlayer import HumanPlayer

In [ ]:
# Set a seed, so randomization is reproducable
ChessAI.random.seed(3)

## Spielablauf
Hier wird ein neues Spiel initialisiert und ein fester Seed definiert. Der statische Seed dient dazu, sämtliche Zufallsfunktionen reproduzierbar zu machen.



In [ ]:
import chess
from IPython.display import clear_output, display
from time import sleep

# Create a new board
board = chess.Board()

# Create Players
player1 = Exercise01AI("Player 1") # White
player2 = Exercise01AI("Player 2") # Black
players = [player1, player2]

# All players start with opening game
previous_state = State.OPENING_GAME

# Show board before first move (for human player)
display(board)

# Play game
while True:
    for player in players:
        # Make next move
        player.make_turn(board)

        # Print board
        clear_output(wait=True)
        display(board)
        print(f'Actual state: {player.state}')
        
        # Watch game progress slowly
        # sleep(0.1)

        # Exit if game has finished
        if player.state == State.FINISHED:
            break
    
    # Log state transitions
    if previous_state != player.state:
        print(f"\n {player.name}: Transition from {previous_state} to {player.state} \n")

        # Exit if game has finished
        if player.state == State.FINISHED:
            print(f"{player.name} lost the match!")
            break

        # Update state
        previous_state = player.state

        # Pause at transition
        # input("Press enter to continue")


## Speichern der Partie
Optional kann die Partie hier im PGN-Format gespeichert werden.

In [ ]:
from datetime import datetime
import chess.pgn

# Saves last game to 'games/YYYY-mm-dd_HH-MM-SS.pgn' (based on current time)

game = chess.pgn.Game.from_board(board)
game.headers["Event"] = "Chess-AI game"
game.headers["Date"] = datetime.now().strftime("%d.%m.%Y")
filename = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
with open(f"../games/{filename}.pgn", 'w') as gamefile:
    gamefile.write(str(game))
